## Step 40: Build SWC Stations Dataset

Purpose: Combine COOP and ISD stations into a single SWC station dataset.

Note: this notebook is relatively short as its expected additional station processing 
logic will be added at a later date.

In [1]:
import arcpy;
import os,sys;
import json,requests;
import datetime,http.client;
import zipfile,shutil,csv;
import time;

print("Step 40: Build SWC Stations Dataset");

def swc_resources():
    
    rez = {};
    
    # Verify or Create Source filegeodatabase
    rez['source'] = os.getcwd() + os.sep + 'source.gdb';

    if not arcpy.Exists(rez['source']):
        print("  creating new source workspace");
        arcpy.CreateFileGDB_management(
             os.path.dirname(rez['source'])
            ,os.path.basename(rez['source'])
        );
        
    # Verify or Create Working filegeodatabase
    rez['working'] = os.getcwd() + os.sep + 'working.gdb';

    if not arcpy.Exists(rez['working']):
        print("  creating new working workspace");
        arcpy.CreateFileGDB_management(
             os.path.dirname(rez['working'])
            ,os.path.basename(rez['working'])
        );
        
    # Verify or Create Results filegeodatabase
    rez['results'] = os.getcwd() + os.sep + 'results.gdb';

    if not arcpy.Exists(rez['results']):
        print("  creating new results workspace");
        arcpy.CreateFileGDB_management(
             os.path.dirname(rez['results'])
            ,os.path.basename(rez['results'])
        );

    # Verify or Create qa directory
    rez['qa'] = os.getcwd() + os.sep + 'qa';

    if not arcpy.Exists(rez['qa']):
        print("  creating new qa directory");
        os.mkdir(rez['qa']);
        
    # Verify or Create resources directory
    rez['resources'] = os.getcwd() + os.sep + 'resources';

    if not arcpy.Exists(rez['resources']):
        print("  creating new resources directory");
        os.mkdir(rez['resources']);
        
    # Verify existence of files directory
    rez['files'] = os.getcwd() + os.sep + 'files';
    
    if not arcpy.Exists(rez['resources']):
        raise Exception('ERROR: project files directory not found');
        
    return rez;

rez = swc_resources();

coop = rez['source'] + os.sep + 'COOP_STATIONS_TO_USE';
coop_cnt = arcpy.GetCount_management(coop)[0];
isd  = rez['source'] + os.sep + 'ISD_STATIONS_TO_USE';
isd_cnt = arcpy.GetCount_management(isd)[0];

print("  COOP Stations    : " + str(coop_cnt));
print("  ISD Stations     : " + str(isd_cnt));


Step 40: Build SWC Stations Dataset
  COOP Stations    : 1851
  ISD Stations     : 3293


### 40.010: Create fresh D4EMLite_PREC_Details working dataset

In [2]:
%%time

wrkstat0 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK';

print("  creating new working stations");
if arcpy.Exists(wrkstat0):
    arcpy.Delete_management(wrkstat0);
  
arcpy.CreateFeatureclass_management(
     out_path      = os.path.dirname(wrkstat0)
    ,out_name      = os.path.basename(wrkstat0)
    ,geometry_type = "POINT"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = wrkstat0
    ,field_description = [
         ['StationId'   ,'TEXT'  ,'StationId'   ,14 ]
        ,['DataType'    ,'TEXT'  ,'DataType'    ,255]
        ,['FileName'    ,'TEXT'  ,'FileName'    ,255]
        ,['Id'          ,'LONG'  ,'Id'              ]
        ,['Lat'         ,'DOUBLE','Lat'             ]
        ,['Long'        ,'DOUBLE','Long'            ]
        ,['Scenario'    ,'TEXT'  ,'Scenario'    ,255]
        ,['Constituent' ,'TEXT'  ,'Constituent' ,255]
        ,['SDate'       ,'TEXT'  ,'SDate'       ,12 ]
        ,['EDate'       ,'TEXT'  ,'EDate'       ,12 ]
        ,['YrCount'     ,'DOUBLE','YrCount'         ]
        ,['Value'       ,'DOUBLE','Value'           ]
        ,['StaNam'      ,'TEXT'  ,'StaNam'      ,255]
    ]
);

print("  adding indexes");
z = arcpy.management.AddIndex(
     in_table   = wrkstat0
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);


  creating new working stations
  adding indexes
Wall time: 2.71 s


### 40.020: Load COOP and ISD station data into new D4EMLite_PREC_Details working feature class

In [3]:
%%time

wrkstat0 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK';
coop     = rez['source'] + os.sep + 'COOP_STATIONS_TO_USE';
isd      = rez['source'] + os.sep + 'ISD_STATIONS_TO_USE';

fldout = [
     'StationId'
    ,'DataType'
    ,'FileName'
    ,'Id'
    ,'Lat'
    ,'Long'
    ,'Scenario'
    ,'Constituent'
    ,'SDate'
    ,'EDate'
    ,'YrCount'
    ,'Value'
    ,'StaNam'
    ,'SHAPE@'
];

fldcoop = [
     'station_id'
    ,'station_name'
    ,'state'
    ,'start_date_clean'
    ,'end_date_clean'
    ,'latitude'
    ,'longitude'
    ,'in_basins'
    ,'break_with_basins'
    ,'network'
    ,'year_count'
    ,'start_date_to_use_clean'
    ,'end_date_to_use_clean'
    ,'SHAPE@'
];

fldisd = [
     'station_id'
    ,'station_name'
    ,'state'
    ,'start_date_clean'
    ,'end_date_clean'
    ,'latitude'
    ,'longitude'
    ,'in_basins'
    ,'break_with_basins'
    ,'network'
    ,'year_count'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(
     in_table    = wrkstat0
    ,field_names = fldout
) as outcur:

    print("  loading COOP stations");
    with arcpy.da.SearchCursor(
         in_table    = coop
        ,field_names = fldcoop
    ) as incur:
        for row in incur:
            
            outcur.insertRow((
                 row[0]
                ,'WdmFinal'
                ,row[2].lower() + str(row[0]) + '.txt'
                ,1
                ,row[5]
                ,row[6]
                ,'OBSERVED'
                ,'PREC'
                ,row[11]
                ,row[12]
                ,row[10]
                ,0.0
                ,row[1]
                ,row[13]
            ));
            
    print("  loading ISD stations");
    with arcpy.da.SearchCursor(
         in_table    = isd
        ,field_names = fldisd
    ) as incur:
        for row in incur:
            
            outcur.insertRow((
                 row[0]
                ,'WdmFinal'
                ,row[2].lower() + str(row[0]) + '.txt'
                ,1
                ,row[5]
                ,row[6]
                ,'OBSERVED'
                ,'PREC'
                ,row[3]
                ,row[4]
                ,row[10]
                ,0.0
                ,row[1]
                ,row[11]
            ));

wrkstat0_cnt = arcpy.GetCount_management(wrkstat0)[0];
print("  working station count: " + str(wrkstat0_cnt));


  loading COOP stations
  loading ISD stations
  working station count: 5144
Wall time: 867 ms


### 40.030: Review and QA

QA Products:

1. station counts by state saved to **step40qa.txt**


In [4]:
%%time

wrkstat0 = rez['working'] + os.sep + 'D4EMLite_PREC_Details_WRK';
stat_cnt = arcpy.GetCount_management(wrkstat0)[0];

print("  New Stations: " + str(stat_cnt));

if arcpy.Exists('memory/stats'):
    arcpy.Delete_management('memory/stats');
    
arcpy.conversion.FeatureClassToFeatureClass(
     in_features  = wrkstat0
    ,out_path     = 'memory'
    ,out_name     = 'stats'
);

arcpy.management.AddFields(
     in_table          = 'memory/stats'
    ,field_description = [
         ['state'     ,'TEXT','state'     ,2]
    ]
);

z = arcpy.management.CalculateField(
     in_table        = 'memory/stats'
    ,field           = 'state'
    ,expression      = "substate(!FileName!)"
    ,expression_type = 'PYTHON3'
    ,code_block      = """
def substate(pin):
    return pin[:2];
"""
);
    
if arcpy.Exists('memory/stats2'):
    arcpy.Delete_management('memory/stats2');
 
arcpy.Statistics_analysis(
     in_table          = 'memory/stats'
    ,out_table         = 'memory/stats2'
    ,statistics_fields = [["FileName","COUNT"]]
    ,case_field        = ['state']
);

with open(rez['qa'] + os.sep + 'step40qa.txt',"w") as out:
    out.write("Step 40 QA Review\n");
    out.write(datetime.datetime.now().isoformat() + "\n");
    out.write("Total Stations," + str(stat_cnt) + "\n");

    with arcpy.da.SearchCursor(
         in_table    = 'memory/stats2'
        ,field_names = ['state','COUNT_FileName']
        ,sql_clause  = (None,'ORDER BY state')
    ) as cur:
        
        for row in cur:
            out.write(row[0] + ',' + str(row[1]) + "\n");


  New Stations: 5144
Wall time: 1.48 s
